In [2]:
import sys
sys.path.append("../")
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
from plotting import CandlePlot

In [3]:
from infrastructure.instrument_collection import instrumentCollection as ic

# Establish Pair

In [32]:
pair = "GBP_JPY"
granularity = "H4"
df = pd.read_pickle(f"../data/{pair}_{granularity}.pkl")
MA_LIST = [10, 20, 50, 100, 200]

In [33]:
df_ma = df[['time','mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [34]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c
0,2022-04-01 13:00:00+00:00,161.017,161.136,160.579,160.629
1,2022-04-01 17:00:00+00:00,160.627,160.762,160.390,160.730
2,2022-04-03 21:00:00+00:00,160.648,160.872,160.284,160.500
3,2022-04-04 01:00:00+00:00,160.502,161.020,160.479,160.819
4,2022-04-04 05:00:00+00:00,160.819,161.177,160.750,160.828


In [35]:
for ma in MA_LIST:
    df_ma[f'MA_{ma}'] = df_ma['mid_c'].rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(inplace=True, drop=True)    

In [36]:
df_ma.head(5)

,time,mid_o,mid_h,mid_l,mid_c,MA_10,MA_20,MA_50,MA_100,MA_200
0,2022-05-18 17:00:00+00:00,158.920,158.920,158.067,158.264,160.5898,159.63740,159.58650,160.91875,162.416520
1,2022-05-18 21:00:00+00:00,158.211,158.788,157.894,158.572,160.3063,159.65095,159.53448,160.87781,162.406235
2,2022-05-19 01:00:00+00:00,158.572,159.818,158.490,159.580,160.0815,159.70610,159.50526,160.86045,162.400485
3,2022-05-19 05:00:00+00:00,159.578,159.665,158.178,158.440,159.7807,159.68420,159.45418,160.83655,162.390185
4,2022-05-19 09:00:00+00:00,158.439,159.084,157.882,158.084,159.4287,159.69210,159.39766,160.81745,162.376510


In [37]:
df_plot = df_ma.iloc[:500]

In [38]:
cp = CandlePlot(df_plot)

In [39]:
traces = [f'MA_{x}' for x in MA_LIST]

In [40]:
cp.show_plot(line_traces=traces)

In [60]:
MA_S = "MA_50"
MA_L = "MA_200"
BUY = 1
SELL = -1
NONE = 0

In [61]:
df_an = df_ma[['time','mid_o', 'mid_h', 'mid_l', 'mid_c', MA_S, MA_L]].copy()

In [62]:
df_an.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200
0,2022-05-18 17:00:00+00:00,158.920,158.920,158.067,158.264,159.58650,162.416520
1,2022-05-18 21:00:00+00:00,158.211,158.788,157.894,158.572,159.53448,162.406235
2,2022-05-19 01:00:00+00:00,158.572,159.818,158.490,159.580,159.50526,162.400485
3,2022-05-19 05:00:00+00:00,159.578,159.665,158.178,158.440,159.45418,162.390185
4,2022-05-19 09:00:00+00:00,158.439,159.084,157.882,158.084,159.39766,162.376510


In [63]:
df_an['DELTA'] = df_an['MA_50'] - df_an['MA_200']
df_an['DELTA_PREV'] = df_an['DELTA'].shift(1)

In [64]:
def is_trade(row):
    if row['DELTA'] >= 0 and row['DELTA_PREV'] < 0:
        return BUY
    elif row['DELTA'] <0 and row['DELTA_PREV'] >=0:
        return SELL
    return NONE

In [65]:
df_an['TRADE'] = df_an.apply(is_trade, axis=1)

In [66]:
df_trades = df_an[df_an['TRADE'] != NONE].copy()

In [67]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE
75,2022-06-06 05:00:00+00:00,163.326,164.459,163.120,164.392,161.60064,161.523375,0.077265,-0.050600,1
220,2022-07-08 09:00:00+00:00,161.999,163.584,161.901,163.529,163.93832,163.947075,-0.008755,0.057265,-1
287,2022-07-25 13:00:00+00:00,164.820,165.093,164.292,164.740,164.69278,164.651470,0.041310,-0.005960,1
314,2022-08-01 01:00:00+00:00,161.767,161.832,161.046,161.542,164.36944,164.414380,-0.044940,0.035605,-1
481,2022-09-07 21:00:00+00:00,165.785,166.325,165.761,166.083,162.52420,162.482745,0.041455,-0.039300,1


In [81]:
cp = CandlePlot(df_an.iloc[70:300])
cp.show_plot(line_traces=[MA_S, MA_L])

In [69]:
ic.LoadInstruments("../data")

In [70]:
ic.instruments_dict[pair]

{'name': 'GBP_JPY', 'ins_type': 'CURRENCY', 'displayName': 'GBP/JPY', 'pipLocation': 0.01, 'tradeUnitsPrecision': 0, 'marginRate': 0.05}

In [71]:
ins_data = ic.instruments_dict[pair]

In [80]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE,DIFF,GAIN,GAIN_C
75,2022-06-06 05:00:00+00:00,163.326,164.459,163.120,164.392,161.60064,161.523375,0.077265,-0.050600,1,-0.863,-86.3,-86.3
220,2022-07-08 09:00:00+00:00,161.999,163.584,161.901,163.529,163.93832,163.947075,-0.008755,0.057265,-1,1.211,-121.1,-207.4
287,2022-07-25 13:00:00+00:00,164.820,165.093,164.292,164.740,164.69278,164.651470,0.041310,-0.005960,1,-3.198,-319.8,-527.2
314,2022-08-01 01:00:00+00:00,161.767,161.832,161.046,161.542,164.36944,164.414380,-0.044940,0.035605,-1,4.541,-454.1,-981.3
481,2022-09-07 21:00:00+00:00,165.785,166.325,165.761,166.083,162.52420,162.482745,0.041455,-0.039300,1,-16.586,-1658.6,-2639.9


In [73]:
df_trades.shape

(23, 10)

In [74]:
df_trades['DIFF'] = df_trades['mid_c'].diff().shift(-1)
df_trades.fillna(0, inplace=True)

In [75]:
df_trades['GAIN'] = df_trades['DIFF'] / ins_data.pipLocation
df_trades['GAIN'] = df_trades['GAIN'] * df_trades['TRADE']

In [76]:
df_trades['GAIN'].sum()

np.float64(-2719.899999999995)

# Plotting Gains over Time

In [78]:
df_trades['GAIN_C'] = df_trades['GAIN'].cumsum()

In [79]:
cp = CandlePlot(df_trades, candles=False)
cp.show_plot(line_traces=['GAIN_C'])